# Homework 3: Sampling
- Name: Congxin (David) Xu
- Computing ID: cx2rx

## Problem 1
(10) Suppose whether it is sunny or not in Charlottesville depends on the
weather of the last three days. Show how this can be modeled as a Markov
chain.